# Retry parser

While in some cases it is possible to fix any parsing mistakes by only looking at the output, in other cases it isn't. An example of this is when the output is not just in the incorrect format, but is partially complete. Consider the below example.

In [3]:
from langchain.output_parsers import (
    OutputFixingParser,
    PydanticOutputParser,
)
from langchain.prompts import (
    PromptTemplate,
)
from langchain_openai import ChatOpenAI, OpenAI
from pydantic import BaseModel, Field

In [4]:
template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""


class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")


parser = PydanticOutputParser(pydantic_object=Action)

In [5]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n Don not forget use double quotes",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [6]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

In [7]:
bad_response = "{'action': 'search'}"

If we try to parse this response as is, we will get an error:

In [8]:
parser.parse(bad_response)

OutputParserException: Failed to parse Action from completion {'action': 'search'}. Got: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

If we try to use the `OutputFixingParser` to fix this error, it will be confused - namely, it doesn't know what to actually put for action input.

In [9]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

Please note that due to model capabilities, errors may occur here. Please try a few more times.

In [11]:
fix_parser.parse(bad_response)

Action(action='search', action_input='input')

Instead, we can use the RetryOutputParser, which passes in the prompt (as well as the original output) to try again to get a better response.

In [12]:
from langchain.output_parsers import RetryWithErrorOutputParser

In [13]:
retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0)
)

In [14]:
retry_parser.parse_with_prompt(bad_response, prompt_value)

Action(action='search', action_input='leo di caprios gf')